In [9]:
import re
import sentencepiece as spm
from tqdm import tqdm

def clean_text(text):
    # Chuyển thành chữ thường
    text = text.lower()
    
    # Loại bỏ tất cả dấu câu ngoại trừ dấu chấm
    text = re.sub(r'[^\w\s.]', '', text)
    
    # Thay thế nhiều khoảng trắng bằng một khoảng trắng duy nhất và loại bỏ khoảng trắng ở đầu/cuối
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Dataset paths
en_file_path = 'TED2020.en-vi.en'
vi_file_path = 'TED2020.en-vi.vi'

# Read and clean data
try:
    with open(en_file_path, 'r', encoding='utf-8') as en_file, \
         open(vi_file_path, 'r', encoding='utf-8') as vi_file, \
         open('cleaned.en', 'w', encoding='utf-8') as en_out, \
         open('cleaned.vi', 'w', encoding='utf-8') as vi_out:
        for en_line, vi_line in zip(en_file, vi_file):
            en_clean = clean_text(en_line)
            vi_clean = clean_text(vi_line)
            if en_clean and vi_clean and len(en_clean.split()) > 1 and len(vi_clean.split()) > 1:
                en_out.write(en_clean + '\n')
                vi_out.write(vi_clean + '\n')
except FileNotFoundError as e:
    print(f"Error: File not found. Check the file paths: {e}")
    exit(1)
except Exception as e:
    print(f"An error occurred during data cleaning: {e}")
    exit(1)

# Combine data for GPT (English -> Vietnamese)
try:
    with open('cleaned.en', 'r', encoding='utf-8') as en_file, \
         open('cleaned.vi', 'r', encoding='utf-8') as vi_file, \
         open('gpt_data.txt', 'w', encoding='utf-8') as out_file:
        for en_line, vi_line in zip(en_file, vi_file):
            combined = f"Source: {en_line.strip()} Target: {vi_line.strip()} [EOS]\n"
            out_file.write(combined)
except Exception as e:
    print(f"Error combining data: {e}")
    exit(1)

In [10]:
import time

# Train SentencePiece with BPE
"""
BPE (Byte Pair Encoding) là một thuật toán nén và token hóa văn bản. 
Cách hoạt động:
1. Bắt đầu với tất cả các ký tự riêng lẻ trong văn bản (bao gồm cả ký hiệu đặc biệt).
2. Lặp lại: Tìm cặp ký tự (hoặc subword) xuất hiện thường xuyên nhất và hợp nhất chúng thành một token mới.
3. Tiếp tục hợp nhất cho đến khi đạt được kích thước từ vựng mong muốn (vocab_size).
4. Kết quả là một từ vựng chứa các subword, giúp xử lý các từ hiếm hoặc từ mới hiệu quả.
SentencePiece áp dụng BPE để tạo ra các token subword, phù hợp với các tác vụ dịch máy.
"""
start_time = time.time()
spm.SentencePieceTrainer.train(
    input='gpt_data.txt',
    model_prefix='ted2020_spm',
    vocab_size=52000,
    character_coverage=1.0,
    model_type='bpe',
    user_defined_symbols=['[EOS]', '[PAD]', '[BOS]', 'Source:', 'Target:'],
    pad_id=0,
    bos_id=1,
    eos_id=2,
    unk_id=3
)
print(f"Training spm took {time.time() - start_time:.2f} seconds")

start_time = time.time()
sp = spm.SentencePieceProcessor(model_file='ted2020_spm.model')
print(f"Loading tokenizer took {time.time() - start_time:.2f} seconds")

# Hiển thị một số token trong từ vựng để minh họa BPE
print("Sample tokens from BPE vocabulary:")
with open('ted2020_spm.vocab', 'r', encoding='utf-8') as vocab_file:
    for i, line in enumerate(vocab_file):
        if i >= 10:  # In 10 token đầu tiên
            break
        token, _ = line.strip().split('\t')
        print(f"Token {i}: {token}")

# Tokenize data
def tokenize_gpt_file(input_file, output_file, sp):
    """
    Hàm này mã hóa văn bản thành các token số nguyên sử dụng BPE.
    Mỗi dòng văn bản được chia thành các subword dựa trên từ vựng BPE đã huấn luyện.
    """
    try:
        with open(input_file, 'r', encoding='utf-8') as f_in:
            lines = f_in.readlines()
        with open(output_file, 'w', encoding='utf-8') as f_out:
            for line in tqdm(lines, desc="Tokenizing data with BPE"):
                tokens = sp.encode(line.strip(), out_type=int)
                valid_tokens = [t for t in tokens if 0 <= t < 32000]
                if valid_tokens:
                    f_out.write(' '.join(map(str, valid_tokens)) + '\n')
    except Exception as e:
        print(f"Error tokenizing data: {e}")
        exit(1)

start_time = time.time()
tokenize_gpt_file('gpt_data.txt', 'tokenized_gpt.txt', sp)
print(f"Tokenizing data took {time.time() - start_time:.2f} seconds")

# Verify special tokens
print(f"Pad ID: {sp.pad_id()}")
print(f"BOS ID: {sp.bos_id()}")
print(f"EOS ID: {sp.eos_id()}")
print(f"UNK ID: {sp.unk_id()}")
print(f"Vocab size: {sp.get_piece_size()}")

# Minh họa cách BPE mã hóa một câu ví dụ
example_sentence = "Source: i like to learn new things Target: tôi thích học những thứ mới [EOS]"
encoded = sp.encode(example_sentence, out_type=str)  # out_type=str để thấy các subword
print(f"Example sentence: {example_sentence}")
print(f"BPE tokenized: {encoded}")

Training spm took 20.18 seconds
Loading tokenizer took 0.04 seconds
Sample tokens from BPE vocabulary:
Token 0: <pad>
Token 1: <s>
Token 2: </s>
Token 3: <unk>
Token 4: [EOS]
Token 5: [PAD]
Token 6: [BOS]
Token 7: Source:
Token 8: Target:
Token 9: ▁t


Tokenizing data with BPE: 100%|██████████| 316928/316928 [00:27<00:00, 11417.64it/s]

Tokenizing data took 28.13 seconds
Pad ID: 0
BOS ID: 1
EOS ID: 2
UNK ID: 3
Vocab size: 52000
Example sentence: Source: i like to learn new things Target: tôi thích học những thứ mới [EOS]
BPE tokenized: ['▁', 'Source:', '▁i', '▁like', '▁to', '▁learn', '▁new', '▁things', '▁', 'Target:', '▁tôi', '▁thích', '▁học', '▁những', '▁thứ', '▁mới', '▁', '[EOS]']
